# SHARK ATTACKS

In [1]:
# import libraries
import pandas as pd
import re
from functions import yn_to_boolean 
from functions import empire

### HYPOTHESIS:
The dangers of World Exploration by Sea.

As we know, during the Age of Discovery from the 15th to the 17th centuries, humanity began exploring the oceans and reached many new lands around the globe. Most of those discoveries and the beginning of globalization would not have been possible without the work and brave spirit of sailors from the Old World, who left their homes to fight many dangers, known and unknown, for the glory and power of their native empires.

In this analysis, we will look at one of those dangers, the scariest of sea monsters, the shark. Whether walking the plank on the hands of a mighty pirate or kite surfing in sunny California, many a man lost their lives to an attack from one of these vicious creatures.

Can we find a relation, though, between these attacks and the zones conquered by the largest european empires? Let's look through the data and find out if sharks are particularly hungry or deadly in one of those territories.

In [2]:
# Read CSV file to DataFrame
# CSV file should be added to input folder
shark_attacks = pd.read_csv('input/GSAF5.csv')

In [3]:
shark_attacks

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [4]:
# Checking for columns with high rates of Null values
print(shark_attacks.shape)
null_col = shark_attacks.isnull().sum()
null_percentage = null_col.apply(lambda x : round(x*100/shark_attacks.shape[0],2))
null_percentage

(5992, 24)


Case Number                0.00
Date                       0.00
Year                       0.00
Type                       0.00
Country                    0.72
Area                       6.71
Location                   8.28
Activity                   8.80
Name                       3.34
Sex                        9.46
Age                       44.74
Injury                     0.45
Fatal (Y/N)                0.32
Time                      53.62
Species                   48.97
Investigator or Source     0.25
pdf                        0.00
href formula               0.02
href                       0.05
Case Number.1              0.00
Case Number.2              0.00
original order             0.00
Unnamed: 22               99.98
Unnamed: 23               99.97
dtype: float64

In [5]:
# Columns 'Unnamed: 22' and 'Unnamed: 23' due to being almost completely null.
# Columns 'Age', 'Time' and 'Species' require further investigation.
shark_attacks.drop(columns=['Unnamed: 22','Unnamed: 23'], inplace=True)
shark_attacks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [6]:
# Comparing 'Case Number', 'Case Number.1' and ''Case Number.2' 
print((shark_attacks['Case Number'] == shark_attacks['Case Number.1']).value_counts())
print((shark_attacks['Case Number'] == shark_attacks['Case Number.2']).value_counts())

True     5979
False      13
dtype: int64
True     5990
False       2
dtype: int64


In [7]:
# Safe to assume 'Case Number.1' and 'Case Number.2' are duplicates of 'Case Number'
shark_attacks.drop(columns=['Case Number.1','Case Number.2'], inplace=True)

In [8]:
shark_attacks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'original order'],
      dtype='object')

In [9]:
# Comparing 'href formula' and 'href'
print((shark_attacks['href formula'] == shark_attacks['href']).value_counts())

True     5938
False      54
dtype: int64


In [10]:
# Also safe to assume duplication
shark_attacks.drop(columns=['href'], inplace=True)

In [11]:
list(shark_attacks.columns)

['Case Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex ',
 'Age',
 'Injury',
 'Fatal (Y/N)',
 'Time',
 'Species ',
 'Investigator or Source',
 'pdf',
 'href formula',
 'original order']

In [12]:
# Renaming columns to avoid naming issues
shark_attacks.rename(columns={'Case Number':'case_number_id',
 'Date':'date',
 'Year':'year',
 'Type':'type_of_attack',
 'Country':'country',
 'Area':'area',
 'Location':'location',
 'Activity':'activity',
 'Name':'name',
 'Sex ':'sex',
 'Age':'age',
 'Injury':'injury',
 'Fatal (Y/N)':'fatal',
 'Time':'time',
 'Species ':'species',
 'Investigator or Source':'source',
 'href formula':'hyperlink',
 'original order': 'original_id'}, inplace=True)

In [13]:
shark_attacks.columns

Index(['case_number_id', 'date', 'year', 'type_of_attack', 'country', 'area',
       'location', 'activity', 'name', 'sex', 'age', 'injury', 'fatal', 'time',
       'species', 'source', 'pdf', 'hyperlink', 'original_id'],
      dtype='object')

In [14]:
# Avaliating fatality column
shark_attacks['fatal'].value_counts()

N          4315
Y          1552
UNKNOWN      94
 N            8
N             1
n             1
#VALUE!       1
F             1
Name: fatal, dtype: int64

In [15]:
# Rows of UNKNOWN or #VALUE should be dropped, as they won't waiger much in our analysis.
# Other results out of pattern shall be corrected and this column makes most sense as Boolean.
shark_attacks = shark_attacks[~shark_attacks.fatal.isin(['UNKNOWN', '#VALUE!'])]
shark_attacks.fatal.value_counts()

N     4315
Y     1552
 N       8
N        1
n        1
F        1
Name: fatal, dtype: int64

In [16]:
# Applying function to change values into boolean
shark_attacks.fatal=shark_attacks.fatal.astype(str).apply(yn_to_boolean).astype(bool)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
shark_attacks.fatal.value_counts()

False    4345
True     1552
Name: fatal, dtype: int64

In [18]:
# Cleaning the values of column country
# Removing unwanted spaces
shark_attacks.country.astype(str).apply(lambda x : x.strip())

0                      USA
1                      USA
2                      USA
3                AUSTRALIA
4                AUSTRALIA
               ...        
5987             AUSTRALIA
5988             AUSTRALIA
5989                   USA
5990                PANAMA
5991    CEYLON (SRI LANKA)
Name: country, Length: 5897, dtype: object

In [19]:
all_countries = ["AFGHANISTAN", "ALBANIA", "ALGERIA", "AMERICAN SAMOA", "ANDORRA", "ANGOLA", "ANGUILLA", "ANTARCTICA", "ANTIGUA AND BARBUDA", "ARGENTINA", "ARMENIA", "ARUBA", "AUSTRALIA", "AUSTRIA", "AZERBAIJAN", "BAHAMAS", "BAHRAIN", "BANGLADESH", "BARBADOS", "BELARUS", "BELGIUM", "BELIZE", "BENIN", "BERMUDA", "BHUTAN", "BOLIVIA", "BOSNIA AND HERZEGOWINA", "BOTSWANA", "BOUVET ISLAND", "BRAZIL", "BRITISH INDIAN OCEAN TERRITORY", "BRUNEI DARUSSALAM", "BULGARIA", "BURKINA FASO", "BURUNDI", "CAMBODIA", "CAMEROON", "CANADA", "CAPE VERDE", "CAYMAN ISLANDS", "CENTRAL AFRICAN REPUBLIC", "CHAD", "CHILE", "CHINA", "CHRISTMAS ISLAND", "COCOS (KEELING) ISLANDS", "COLOMBIA", "COMOROS", "CONGO", "CONGO, THE DEMOCRATIC REPUBLIC OF THE", "COOK ISLANDS", "COSTA RICA", "COTE D'IVOIRE", "CROATIA", "CUBA", "CYPRUS", "CZECH REPUBLIC", "DENMARK", "DJIBOUTI", "DOMINICA", "DOMINICAN REPUBLIC", "EAST TIMOR", "ECUADOR", "EGYPT", "EL SALVADOR", "EQUATORIAL GUINEA", "ERITREA", "ESTONIA", "ETHIOPIA", "FALKLAND ISLANDS (MALVINAS)", "FAROE ISLANDS", "FIJI", "FINLAND", "FRANCE", "FRANCE METROPOLITAN", "FRENCH GUIANA", "FRENCH POLYNESIA", "FRENCH SOUTHERN TERRITORIES", "GABON", "GAMBIA", "GEORGIA", "GERMANY", "GHANA", "GIBRALTAR", "GREECE", "GREENLAND", "GRENADA", "GUADELOUPE", "GUAM", "GUATEMALA", "GUINEA", "GUINEA-BISSAU", "GUYANA", "HAITI", "HEARD AND MC DONALD ISLANDS", "HOLY SEE (VATICAN CITY STATE)", "HONDURAS", "HONG KONG", "HUNGARY", "ICELAND", "INDIA", "INDONESIA", "IRAN", "IRAQ", "IRELAND", "ISRAEL", "ITALY", "JAMAICA", "JAPAN", "JORDAN", "KAZAKHSTAN", "KENYA", "KIRIBATI", "NORTH KOREA", "SOUTH KOREA", "KUWAIT", "KYRGYZSTAN", "LAO, PEOPLE'S DEMOCRATIC REPUBLIC", "LATVIA", "LEBANON", "LESOTHO", "LIBERIA", "LIBYA", "LIECHTENSTEIN", "LITHUANIA", "LUXEMBOURG", "MACAU", "MACEDONIA, THE FORMER YUGOSLAV REPUBLIC OF", "MADAGASCAR", "MALAWI", "MALAYSIA", "MALDIVES", "MALI", "MALTA", "MARSHALL ISLANDS", "MARTINIQUE", "MAURITANIA", "MAURITIUS", "MAYOTTE", "MEXICO", "MICRONESIA, FEDERATED STATES OF", "MOLDOVA, REPUBLIC OF", "MONACO", "MONGOLIA", "MONTSERRAT", "MOROCCO", "MOZAMBIQUE", "MYANMAR", "NAMIBIA", "NAURU", "NEPAL", "NETHERLANDS", "NETHERLANDS ANTILLES", "NEW CALEDONIA", "NEW ZEALAND", "NICARAGUA", "NIGER", "NIGERIA", "NIUE", "NORFOLK ISLAND", "NORTHERN MARIANA ISLANDS", "NORWAY", "OMAN", "PAKISTAN", "PALAU", "PANAMA", "PAPUA NEW GUINEA", "PARAGUAY", "PERU", "PHILIPPINES", "PITCAIRN", "POLAND", "PORTUGAL", "PUERTO RICO", "QATAR", "REUNION", "ROMANIA", "RUSSIA", "RWANDA", "SAINT KITTS AND NEVIS", "SAINT LUCIA", "SAINT VINCENT AND THE GRENADINES", "SAMOA", "SAN MARINO", "SAO TOME AND PRINCIPE", "SAUDI ARABIA", "SENEGAL", "SEYCHELLES", "SIERRA LEONE", "SINGAPORE", "SLOVAKIA (SLOVAK REPUBLIC)", "SLOVENIA", "SOLOMON ISLANDS", "SOMALIA", "SOUTH AFRICA", "SOUTH GEORGIA AND THE SOUTH SANDWICH ISLANDS", "SPAIN", "SRI LANKA", "ST. HELENA", "ST. PIERRE AND MIQUELON", "SUDAN", "SURINAME", "SVALBARD AND JAN MAYEN ISLANDS", "SWAZILAND", "SWEDEN", "SWITZERLAND", "SYRIAN ARAB REPUBLIC", "TAIWAN", "TAJIKISTAN", "TANZANIA", "THAILAND", "TOGO", "TOKELAU", "TONGA", "TRINIDAD AND TOBAGO", "TUNISIA", "TURKEY", "TURKMENISTAN", "TURKS AND CAICOS ISLANDS", "TUVALU", "UGANDA", "UKRAINE", "UNITED ARAB EMIRATES", "UNITED KINGDOM", "USA", "UNITED STATES MINOR OUTLYING ISLANDS", "URUGUAY", "UZBEKISTAN", "VANUATU", "VENEZUELA", "VIETNAM", "VIRGIN ISLANDS (BRITISH)", "VIRGIN ISLANDS (U.S.)", "WALLIS AND FUTUNA ISLANDS", "WESTERN SAHARA", "YEMEN", "YUGOSLAVIA", "ZAMBIA", "ZIMBABWE"]
# Filtering values through list of all countries.
shark_attacks.country[~shark_attacks.country.isin(all_countries)].value_counts()

ENGLAND                           19
PACIFIC OCEAN                     17
ATLANTIC OCEAN                    16
SOUTH ATLANTIC OCEAN              12
NEW GUINEA                        10
                                  ..
NORTHERN ARABIAN SEA               1
IRAN / IRAQ                        1
FEDERATED STATES OF MICRONESIA     1
St Helena                          1
DIEGO GARCIA                       1
Name: country, Length: 87, dtype: int64

In [20]:
# Replace mispelled country names
shark_attacks.country=shark_attacks.country.replace("ENGLAND","UNITED KINGDOM").replace("NEW GUINEA","PAPUA NEW GUINEA").replace("COLUMBIA","COLOMBIA").replace("SCOTLAND","UNITED KINGDOM")
shark_attacks.country=shark_attacks.country.replace("Fiji","FIJI").replace("ST. MARTIN","NETHERLANDS").replace("ST. MAARTIN","NETHERLANDS").replace("TRINIDAD & TOBAGO", "TRINIDAD AND TOBAGO")
shark_attacks.country=shark_attacks.country.replace("ANTIGUA", "ANTIGUA AND BARBUDA").replace("TURKS & CAICOS", "UNITED KINGDOM").replace("UNITED ARAB EMIRATES (UAE)","UNITED ARAB EMIRATES")
shark_attacks.country=shark_attacks.country.replace("AZORES","PORTUGAL").replace("BRITISH VIRGIN ISLANDS","UNITED KINGDOM").replace("NEVIS","SAINT KITTS AND NEVIS")
shark_attacks.country=shark_attacks.country.replace("Sierra Leone","SIERRA LEONE").replace("GRAND CAYMAN", "UNITED KINGDOM").replace("OKINAWA","JAPAN").replace("BRITISH ISLES", "UNITED KINGDOM")
shark_attacks.country=shark_attacks.country.replace("BAHREIN", "BAHRAIN").replace("CEYLON (SRI LANKA)", "SRI LANKA").replace("Seychelles", "SEYCHELLES")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
shark_attacks.country[~shark_attacks.country.isin(all_countries)].value_counts().head()

PACIFIC OCEAN           17
ATLANTIC OCEAN          16
SOUTH ATLANTIC OCEAN    12
CARIBBEAN SEA            8
NORTH PACIFIC OCEAN      7
Name: country, dtype: int64

In [22]:
# Dropping further results for inprecise location and statistical irrelevance
shark_attacks = shark_attacks[shark_attacks.country.isin(all_countries)]

In [23]:
set(shark_attacks.country.values)

{'ALGERIA',
 'AMERICAN SAMOA',
 'ANGOLA',
 'ANTIGUA AND BARBUDA',
 'ARGENTINA',
 'ARUBA',
 'AUSTRALIA',
 'BAHAMAS',
 'BAHRAIN',
 'BANGLADESH',
 'BARBADOS',
 'BELIZE',
 'BERMUDA',
 'BRAZIL',
 'CANADA',
 'CAPE VERDE',
 'CAYMAN ISLANDS',
 'CHILE',
 'CHINA',
 'COLOMBIA',
 'COOK ISLANDS',
 'COSTA RICA',
 'CROATIA',
 'CUBA',
 'CYPRUS',
 'DJIBOUTI',
 'DOMINICAN REPUBLIC',
 'ECUADOR',
 'EGYPT',
 'EL SALVADOR',
 'FIJI',
 'FRANCE',
 'FRENCH POLYNESIA',
 'GABON',
 'GEORGIA',
 'GHANA',
 'GREECE',
 'GREENLAND',
 'GRENADA',
 'GUAM',
 'GUATEMALA',
 'GUINEA',
 'GUYANA',
 'HAITI',
 'HONDURAS',
 'HONG KONG',
 'ICELAND',
 'INDIA',
 'INDONESIA',
 'IRAN',
 'IRAQ',
 'IRELAND',
 'ISRAEL',
 'ITALY',
 'JAMAICA',
 'JAPAN',
 'KENYA',
 'KIRIBATI',
 'KUWAIT',
 'LEBANON',
 'LIBERIA',
 'LIBYA',
 'MADAGASCAR',
 'MALAYSIA',
 'MALTA',
 'MARSHALL ISLANDS',
 'MARTINIQUE',
 'MAURITIUS',
 'MAYOTTE',
 'MEXICO',
 'MONACO',
 'MOZAMBIQUE',
 'NAMIBIA',
 'NETHERLANDS',
 'NETHERLANDS ANTILLES',
 'NEW CALEDONIA',
 'NEW ZEALAND',
 

In [24]:
# Sorting in Empires
empires = {'ALGERIA':'FRANCE',
 'ANGOLA':'PORTUGAL',
 'ANTIGUA AND BARBUDA':'UNITED KINGDOM',
 'ARGENTINA':'SPAIN',
 'AUSTRALIA':'UNITED KINGDOM',
 'BAHAMAS':'UNITED KINGDOM',
 'BAHRAIN':'UNITED KINGDOM',
 'BARBADOS':'UNITED KINGDOM',
 'BELIZE':'UNITED KINGDOM',
 'BERMUDA':'UNITED KINGDOM',
 'BRAZIL':'PORTUGAL',
 'CANADA':'UNITED KINGDOM',
 'CAPE VERDE':'PORTUGAL',
 'CAYMAN ISLANDS':'UNITED KINGDOM',
 'CHILE':'SPAIN',
 'COLOMBIA':'SPAIN',
 'COOK ISLANDS':'UNITED KINGDOM',
 'COSTA RICA':'SPAIN',
 'CUBA':'SPAIN',
 'CYPRUS':'UNITED KINGDOM',
 'DJIBOUTI':'FRANCE',
 'DOMINICAN REPUBLIC':'SPAIN',
 'ECUADOR':'SPAIN',
 'EGYPT':'UNITED KINGDOM',
 'EL SALVADOR':'SPAIN',
 'FIJI':'UNITED KINGDOM',
 'FRANCE':'FRANCE',
 'FRENCH POLYNESIA':'FRANCE',
 'GABON':'FRANCE',
 'GHANA':'UNITED KINGDOM',
 'GRENADA':'UNITED KINGDOM',
 'GUAM':'SPAIN',
 'GUATEMALA':'SPAIN',
 'GUINEA':'FRANCE',
 'GUYANA':'UNITED KINGDOM',
 'HAITI':'FRANCE',
 'HONDURAS':'SPAIN',
 'HONG KONG':'UNITED KINGDOM',
 'INDIA':'UNITED KINGDOM',
 'IRAQ':'UNITED KINGDOM',
 'IRELAND':'UNITED KINGDOM',
 'ISRAEL':'UNITED KINGDOM',
 'JAMAICA':'UNITED KINGDOM',
 'KENYA':'UNITED KINGDOM',
 'KIRIBATI':'UNITED KINGDOM',
 'KUWAIT':'UNITED KINGDOM',
 'LEBANON':'FRANCE',
 'MADAGASCAR':'FRANCE',
 'MALAYSIA':'UNITED KINGDOM',
 'MALTA':'UNITED KINGDOM',
 'MARTINIQUE':'FRANCE',
 'MAURITIUS':'UNITED KINGDOM',
 'MAYOTTE':'FRANCE',
 'MEXICO':'SPAIN',
 'MONACO':'FRANCE',
 'MOZAMBIQUE':'PORTUGAL',
 'NAMIBIA':'UNITED KINGDOM',
 'NEW CALEDONIA':'FRANCE',
 'NEW ZEALAND':'UNITED KINGDOM',
 'NICARAGUA':'SPAIN',
 'NIGERIA':'UNITED KINGDOM',
 'PANAMA':'SPAIN',
 'PARAGUAY':'SPAIN',
 'PHILIPPINES':'SPAIN',
 'PORTUGAL':'PORTUGAL',
 'PUERTO RICO':'SPAIN',
 'REUNION':'FRANCE',
 'SAINT KITTS AND NEVIS':'UNITED KINGDOM',
 'SENEGAL':'FRANCE',
 'SEYCHELLES':'FRANCE',
 'SIERRA LEONE':'UNITED KINGDOM',
 'SINGAPORE':'UNITED KINGDOM',
 'SOLOMON ISLANDS':'UNITED KINGDOM',
 'SOMALIA':'UNITED KINGDOM',
 'SOUTH AFRICA':'UNITED KINGDOM',
 'SPAIN':'SPAIN',
 'SRI LANKA':'UNITED KINGDOM',
 'TANZANIA':'UNITED KINGDOM',
 'TONGA':'UNITED KINGDOM',
 'TRINIDAD AND TOBAGO':'UNITED KINGDOM',
 'TUNISIA':'FRANCE',
 'TUVALU':'UNITED KINGDOM',
 'UNITED ARAB EMIRATES':'UNITED KINGDOM',
 'UNITED KINGDOM':'UNITED KINGDOM',
 'URUGUAY':'SPAIN',
 'USA':'UNITED KINGDOM',
 'VANUATU':'FRANCE',
 'VENEZUELA':'SPAIN',
 'VIETNAM':'FRANCE'}

In [25]:
shark_attacks['empire'] = pd.DataFrame(shark_attacks.country.apply(lambda x : empire(x,empires)))
shark_attacks = shark_attacks[['case_number_id','country','empire','fatal','area','location','activity','injury','type_of_attack']]
shark_attacks.to_csv('output/shark_attacks.csv')
shark_attacks.head()

,case_number_id,country,empire,fatal,area,location,activity,injury,type_of_attack
0,2016.09.18.c,USA,UNITED KINGDOM,False,Florida,"New Smyrna Beach, Volusia County",Surfing,Minor injury to thigh,Unprovoked
1,2016.09.18.b,USA,UNITED KINGDOM,False,Florida,"New Smyrna Beach, Volusia County",Surfing,Lacerations to hands,Unprovoked
2,2016.09.18.a,USA,UNITED KINGDOM,False,Florida,"New Smyrna Beach, Volusia County",Surfing,Lacerations to lower leg,Unprovoked
3,2016.09.17,AUSTRALIA,UNITED KINGDOM,False,Victoria,Thirteenth Beach,Surfing,Struck by fin on chest & leg,Unprovoked
4,2016.09.15,AUSTRALIA,UNITED KINGDOM,False,Victoria,Bells Beach,Surfing,No injury: Knocked off board by shark,Unprovoked


In [26]:
# Grouping and adding to find the empire with the deadliest sharks
sharks_by_empire_sum_fatal = shark_attacks[['empire','fatal']].groupby('empire').sum()
sharks_by_empire_count_attacks = shark_attacks[['empire','fatal']].groupby('empire').count()
sharks_by_empire = pd.merge(sharks_by_empire_sum_fatal, sharks_by_empire_count_attacks, on='empire')
sharks_by_empire.columns = ['fatal','total']
sharks_by_empire['lethality'] = round(100*sharks_by_empire.fatal/sharks_by_empire.total,2)
sharks_by_empire.sort_values('fatal', ascending=False)

,fatal,total,lethality
empire,,,
UNITED KINGDOM,940.0,4541,20.70
OTHER,194.0,457,42.45
SPAIN,169.0,329,51.37
FRANCE,87.0,211,41.23
PORTUGAL,64.0,155,41.29


#### Great Britain holds both the highest number of attacks and fatal attacks.
#### However, that is only in absolute numbers. In fact, only a small percentage of the registered shark attacks in those territories resulted in death. 
#### In relative terms, Spain holds the highest lethality, over half of the attacks were lethal.

In [27]:
sharks_by_country_fatal = shark_attacks[['country','empire','fatal']].groupby(['empire','country']).sum()
sharks_by_country_total = shark_attacks[['country','empire','fatal']].groupby(['empire','country']).count()
sharks_by_country = pd.merge(sharks_by_country_fatal,sharks_by_country_total,on=['empire','country'])
sharks_by_country.columns = ['fatal', 'total']
sharks_by_country['lethality'] = round(100*sharks_by_country.fatal/sharks_by_country.total,2)
sharks_by_country.sort_values('fatal', ascending=False).head()

fatal  total  lethality
empire         country                                  
UNITED KINGDOM AUSTRALIA         341.0   1252      27.24
               USA               246.0   2090      11.77
               SOUTH AFRICA      137.0    559      24.51
OTHER          PAPUA NEW GUINEA   62.0    141      43.97
SPAIN          MEXICO             43.0     81      53.09

#### The United Kingdom also takes the lead for absolute numbers of attacks and lethal attacks for a single country.

In [28]:
sharks_by_country.where(sharks_by_country.total>1).sort_values('lethality', ascending=False).head()

fatal  total  lethality
empire country                                
OTHER  LIBERIA           3.0    3.0     100.00
FRANCE GUINEA            3.0    3.0     100.00
OTHER  YEMEN             2.0    2.0     100.00
       AMERICAN SAMOA    3.0    3.0     100.00
SPAIN  NICARAGUA         5.0    6.0      83.33

#### For lethality in single country for countries with more than 1 registered attack, Liberia and Guinea take the lead.

In [29]:
# Looking for outliers
# Create statistics description table of sharks_by_country
stats = sharks_by_country.describe().transpose()
# Adding Interquartile Range column
stats['IQR'] = stats['75%'] - stats['25%']
# Display
stats

,count,mean,std,min,25%,50%,75%,max,IQR
fatal,121.0,12.016529,40.229339,0.0,1.00,2.00,6.00,341.0,5.0
total,121.0,47.049587,225.092827,1.0,2.00,6.00,15.00,2090.0,13.0
lethality,121.0,47.170496,33.630480,0.0,21.43,47.37,69.23,100.0,47.8


In [30]:
# Create outliers table with same columns as sharks_by_country
outliers = pd.DataFrame(columns=sharks_by_country.columns)

In [31]:
# Iterate columns
for col in ['fatal', 'total', 'lethality']:
    # Take IQR value
    iqr = stats.loc[col,'IQR']
    # Set lower and upper outliers bounds
    cutoff = iqr * 1.5
    lower = stats.loc[col,'25%'] - cutoff
    upper = stats.loc[col,'75%'] + cutoff
    # Copy Rows with values of iterating column below lower cut or above upper cut
    results = sharks_by_country[(sharks_by_country[col] < lower) | (sharks_by_country[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)
display(outliers.head())

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7116: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Outlier,fatal,lethality,total
"(FRANCE, NEW CALEDONIA)",fatal,19.0,37.25,51
"(FRANCE, REUNION)",fatal,27.0,47.37,57
"(OTHER, CROATIA)",fatal,18.0,56.25,32
"(OTHER, GREECE)",fatal,15.0,60.00,25
"(OTHER, INDONESIA)",fatal,14.0,70.00,20


In [32]:
# Dropping outliers
sharks_by_country_no_outliers = sharks_by_country.copy()
sharks_by_country_no_outliers.drop(list(outliers.index), inplace=True)
sharks_by_country_no_outliers.sort_values('fatal', ascending=False).head()

fatal  total  lethality
empire         country                                 
OTHER          IRAN              13.0     29      44.83
UNITED KINGDOM EGYPT             13.0     34      38.24
               JAMAICA           13.0     23      56.52
               SOLOMON ISLANDS   12.0     28      42.86
FRANCE         VANUATU            9.0     13      69.23

In [33]:
# Checking for results with outliers removed
sharks_by_empire_no_outliers = sharks_by_country_no_outliers.groupby('empire').sum().drop(columns='lethality')
sharks_by_empire_no_outliers['lethality']=round(100*sharks_by_empire_no_outliers.fatal/sharks_by_empire_no_outliers.total,2)
sharks_by_empire_no_outliers.sort_values('fatal', ascending=False)

,fatal,total,lethality
empire,,,
UNITED KINGDOM,109.0,255,42.75
OTHER,49.0,132,37.12
FRANCE,41.0,103,39.81
SPAIN,37.0,78,47.44
PORTUGAL,4.0,12,33.33


#### Even with outliers removed, result by empire continues similar, Great Britain ahead in absolute numbers and Spain is first in lethality